In [2]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../Configuration"

import refinitiv.data as rd
from refinitiv.data.content import symbol_conversion
import pandas as pd
import numpy as np
from datetime import datetime
import mysql.connector as mysql

import utils.fidatabase as fiutil
import utils.SPYData as spyutil



In [3]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7f85fa91c610 {name='workspace'}>

In [4]:

rics_of_interest = spyutil.getSPYList(rd, pd.to_datetime("2023-04-05").date())


print(rics_of_interest)

Joined twice: HET.N^A08
Deleting added ref HET.N^A08
Joined twice: ROK.N
Deleting added ref ROK.N
Joined twice: MII.N^F99
Deleting added ref MII.N^F99
Joined twice: FJ.N^K00
Deleting added ref FJ.N^K00
Joined twice: RX.N^C10
Deleting added ref RX.N^C10
Joined twice: MWW.N^K16
Deleting added ref MWW.N^K16
Joined twice: PEG.N
Deleting added ref PEG.N
Joined twice: PARA.OQ
Deleting added ref PARA.OQ
Joined twice: JCI.N
Deleting added ref JCI.N
Not found to remove!:  FTI.N
['SO.N', 'KMB.N', 'GIS.N', 'INTC.OQ', 'MSI.N', 'DTE.N', 'UNH.N', 'ETR.N', 'CSCO.OQ', 'IFF.N', 'HAL.N', 'PKI.N', 'HSY.N', 'BBWI.N', 'KR.N', 'MO.N', 'PFE.N', 'MMC.N', 'CMI.N', 'WBA.OQ', 'NUE.N', 'NEM.N', 'WHR.N', 'F.N', 'TGT.N', 'NKE.N', 'PPG.N', 'KO.N', 'PEP.OQ', 'VFC.N', 'NOC.N', 'HD.N', 'DUK.N', 'VZ.N', 'NEE.N', 'CI.N', 'ADSK.OQ', 'CAG.N', 'MDT.N', 'EMN.N', 'GWW.N', 'EMR.N', 'ED.N', 'ORCL.N', 'DOV.N', 'CVX.N', 'MAS.N', 'TJX.N', 'SHW.N', 'AEP.OQ', 'SNA.N', 'ADM.N', 'BAX.N', 'JNJ.N', 'PNC.N', 'IPG.N', 'OXY.N', 'KEY.N', 'T

In [5]:
ric_count = len(rics_of_interest)
day_count = int(.97*(50000/ric_count))


start_date = pd.Timestamp(year=2000,day=1,month=1).date()
end_date =  (start_date + pd.DateOffset(days=day_count)).date()
end_date_final = pd.to_datetime("2023-04-05").date()

print(start_date)
print(day_count)

2000-01-01
96


In [6]:
output_df = pd.DataFrame()

# while end_date < end_date_final:
#     print("Getting data from", start_date, "to", end_date)
#     df = rd.get_history(
#         universe=rics_of_interest,
#         fields=['TR.PriceClose(Curn=USD)'],
#         interval="1D",
#         start=start_date,
#         end=end_date,

#     )
#     output_df = output_df.append(df)
#     start_date = (end_date + pd.DateOffset(days=1)).date()
#     end_date =  (start_date + pd.DateOffset(days=day_count)).date()
#     if(end_date > end_date_final):
#         print("Adjusting end date to", end_date_final, "from", end_date)
#         end_date = end_date_final

#     return_len = len(df)
#     total_len = len(output_df)
#     print("Got", return_len, "rows, total is now", total_len)
#     if( start_date > end_date ):
#         print("Start date Greater than end date - Breaking")
#         break

#     continue



In [9]:
# Write to CSV the output_df dataframe as files.csv
db = fiutil.ConnectToDb()
dbs = db.cursor()


Connected to: 8.0.32


In [10]:
# Read file "S&P500PriceHistory.csv
# The file includes a header row
# Intrepret dates as dates
# The date column is the index

output_df = pd.read_csv("S&P500PriceHistory.csv", parse_dates=['Date'], index_col='Date')


In [19]:
output_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5989 entries, 2000-01-03 to 2023-02-07
Columns: 503 entries, HAS.OQ to FICO.N
dtypes: float64(503)
memory usage: 23.2 MB


In [17]:
index = 0
col_count = 0

for col in output_df.columns:
    print(f"Processing column {col} ({col_count} of {len(output_df.columns)})")


    col_count = col_count + 1
    sql = ""
    to_write = output_df[col].copy()
    to_write.dropna(inplace=True)

    first_col = 0
    last_col = len(to_write)
    
    
    chunk_size = 10000
    
    
    for i in range(first_col, last_col, chunk_size):
        sql = "INSERT INTO TimeSeries ( ric, series, variant, ddate, val ) VALUES (%s, %s, %s, %s)" 
        my_data = []

        next_chunk = min( last_col, chunk_size+i)
        for j in range(i, next_chunk):

            date_val = str(to_write.index[j].date())
            value_val = str(to_write[j])
            my_data.append( ( col, "Price", "RDP", date_val, value_val) ) 

            # sql = sql + f'("{col}", "Price", "RDP", "{date_val}", "{value_val}")'
            # if( j < (next_chunk-1) ):
            #     sql = sql + ","
            # else:
            #     sql = sql + ";"

            # sql = sql + "\n"
    
       
        ## Execute the sql statement stored in variable sql
        dbs.executemany( sql, my_data )
   
       
       

     
 


Processing column HAS.OQ (0 of 503)


Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/Users/eugenemesgar/opt/anaconda3/envs/dev/lib/python3.7/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 1976, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "/Users/eugenemesgar/opt/anaconda3/envs/dev/lib/python3.7/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2011, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
df_dict = {}

for ric in rics_of_interest:
    df_dict[ric] = df[ric].dropna()
    test_df =  df_dict[ric].index.duplicated()
    df_dict[ric] = df_dict[ric].loc[~test_df]
    df_dict[(ric + "PC")] = df_dict[ric].pct_change()
    df_dict[(ric + "CP")] = (df_dict[ric +"PC"] +1).cumprod()
    
graph_start = pd.Timestamp(start_date)
graph_end = (start_date + pd.DateOffset(years=5))

ymax=0
for ric in rics_of_interest:
    ymax = max( ymax, df_dict[(ric + "CP")].loc[graph_start:graph_end].max() )

for ric in rics_of_interest:
    df_dict[(ric + "CP")].plot(legend=True, figsize=(15,10), title="Cumulative Returns", label=ric, xlim=(graph_start,graph_end),ylim=(0,ymax*1.05))



In [ ]:

# for ric in rics_of_interest:
#     ymax = max( ymax, df_dict[(ric + "CP")].loc[graph_start:graph_end].max() )

for ric in rics_of_interest:
    df_dict[(ric + "PC")].plot.hist(bins=100, legend=True, figsize=(15,10), title="Daily Returns", label=ric)
        #legend=True, figsize=(15,10), title="Cumulative Returns", label=ric, xlim=(graph_start,graph_end),ylim=(0,ymax*1.05))


In [ ]:
df_list = list()
for ric in rics_of_interest:
    df_list.append(df_dict[(ric + "PC")])


new_df = pd.concat( df_list , axis=1, join="outer" ) 


new_df.plot.box(legend=True, figsize=(15,10), title="Daily Returns", label=ric)


In [ ]:
new_df.describe().transpose()["std"].sort_values()

In [ ]:
new_df.describe().transpose()["mean"].sort_values()

In [ ]:
sharpe_dict = {}

for ric in rics_of_interest:
    #date_dict[ric + "S"] = df_dict[(ric + "CP")].index[0]
    #date_dict[ric + "E"] = df_dict[(ric + "CP")].index[-1]
    sharpe_dict[ric + "Years"] = (df_dict[(ric + "CP")].index[-1] -  df_dict[(ric + "CP")].index[0]  ).days/365
    sharpe_dict[ric + "Return"] = df_dict[(ric + "CP")].iloc[-1]
    sharpe_dict[ric + "Std"] = df_dict[(ric + "PC")].std() * np.sqrt(252)
    sharpe_dict[ric + "Cagr2"] = (sharpe_dict[ric + "Return"]**(1/sharpe_dict[ric + "Years"])) -  1
    sharpe_dict[ric + "Cagr"] = df_dict[ric + "PC"].mean() *  252
    sharpe_dict[ric + "Sharpe"] =  sharpe_dict[ric+ "Cagr"] / sharpe_dict[ric + "Std"]
    print(f"RIC: {ric} Std: {(sharpe_dict[ric + 'Std']):.2f} Cagr: {(sharpe_dict[ric + 'Cagr']):.2f} CagrA: {(sharpe_dict[ric + 'Cagr2']):.2f} Sharpe: {(sharpe_dict[ric + 'Sharpe']):.2f}")
    #print(ric, sharpe_dict[ric + "Years"], sharpe_dict[ric + "Return"], sharpe_dict[ric + "Std"], sharpe_dict[ric + "Cagr"], sharpe_dict[ric + "Sharpe"] )
                                   

              

In [ ]:
print(sharpe_dict)

In [ ]:
start_date = pd.Timestamp(year=1980,day=1,month=1).date()
end_date = pd.Timestamp(year=2023,day=4,month=4).date()

tdf = rd.get_history( universe=["MSFT.O"], 
                    fields=["TR.F.TotRevenue(Period=LTM,RollPeriods=True)"],
                    interval="1D",
                    start=start_date,
                    end=end_date)

tdf.head()






In [ ]:
tdf.plot(figsize=[13,8], title="MSFT Revenue")


In [ ]:

start_date = pd.Timestamp(year=2000,day=1,month=1).date()
end_date = pd.Timestamp(year=2023,day=4,month=4).date()

tdf = rd.get_history( universe=["MSFT.O"], 
                    fields=["TR.CombinedAlphaRegionRank"],
                    interval="1D",
                    start=start_date,
                    end=end_date)



In [ ]:

display(tdf.tail())
display(tdf.head())

In [ ]:
tdf.plot(figsize=[13,8], title="MSFT CAM")


In [ ]:
cam_and_price = pd.concat( [tdf, df_dict["MSFT.O"]], axis=1, join="outer" )

cam_and_price.columns = ["CAM", "Price"]
cam_and_price["Shift"] = cam_and_price["Price"].pct_change(periods=252)

ax = cam_and_price["CAM"].plot(figsize=[13,8], title="MSFT CAM and Price")
cam_and_price["Shift"].plot(ax=ax, secondary_y=True, style="g")
